In [1]:
#CONFIG
print("Settings.py should be e.g. crops")
print("Still in development, may have bugs")
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
import numpy as np

from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from PIL import Image

c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
#c.config['people_selection'] = range(1,2)
#c.config['body_keypoints_include_list'] = ['LAnkle','RAnkle','LKnee','RKnee','LHip','RHip', 
#                                           'LHeel', 'RHeel,', 'LBigToe', 'RBigToe', 'LSmallToe', 'RBigToe']



Config should be e.g. crops
loading configuration  crops
[OK]


In [2]:
dataset = CasiaDataset()
#for i in range(len(dataset)):
x_max_list = []
y_max_list = []
invalid_pose_counter = 0
print("Using",dataset[0]['poses'].shape[0],"poses")
for item in dataset: 
#for i in range(5):
    #for listitem in dataset: print(listitem)
    annotations = item['annotations']
    scenes = item['scenes']
    poses = item['poses']
#     print("Nr scenes, poses:", len(scenes),poses.shape)
    #print("Currently in", dataset.dataset_items[i])
    #print("Shapes:")
    #print(poses.shape)
    x_tmp = []
    y_tmp = []
    for scene,nr in zip(scenes,range(poses.shape[2])):
        pose = poses[:,:,nr]
        p_x = pose[:,0]
        p_y = pose[:,1]
#         print(p_x)
#         print(p_y)
        if (all(p_x == 0) or all(p_y == 0)):
            print("Problematic pose")
        else:
            x_max = np.max([p for p in p_x if p != 0])
            x_min = np.min([p for p in p_x if p != 0])
            abs_delta_x = x_max-x_min
            x_max_list.append(abs_delta_x)
            y_max = np.max([p for p in p_y if p != 0])
            y_min = np.min([p for p in p_y if p != 0])
            abs_delta_y = y_max-y_min
            y_max_list.append(abs_delta_y) 
            if (abs_delta_x > 640):
                print("Wrong pose",pose)
                
crop_max = [np.ceil(max(x_max_list)), np.ceil(max(y_max_list))]
print("Crop size as double:",max(x_max_list),max(y_max_list))
print("Crop size without margin:",crop_max)


Using 12 poses


KeyboardInterrupt: 

In [ ]:
dataset = CasiaDataset()
x_max = []
y_max = []
x_center = []
y_center = []

margin = 10
print("Using", poses.shape[0],"poses")
invalid_pose_counter = 0
im_size_total = [int(crop_max[1] + 2*margin), int(crop_max[0] + 2*margin)]

#print("Dataset entries", len(dataset.dataset_items[:])

for item, i in zip(dataset, range(len(dataset))):  
    annotations = item['annotations']
    scenes = item['scenes']
    poses = item['poses']
    
    person = '{:03d}'.format(dataset.dataset_items[i][0]) 
    sequence = dataset.dataset_items[i][1]
    angle = '{:03d}'.format(dataset.dataset_items[i][2])
    origin = '/mnt/DATA/HIWI/IBT/CASIA/'
    folderpath = origin + 'images/'+ person + '/'+ sequence + '/' + angle + '/'
    print("current folder: ", folderpath)
    pathlib.Path(origin + 'preprocessing/crops/'+ person + '/' + sequence + '/' + sequence + '-' + angle + '/'
).mkdir(parents=True, exist_ok = True)
    

    #print("Shapes:")
    #print(poses.shape)
    x_tmp = []
    y_tmp = []
#     print("poses shape", poses.shape)
    for j in range(poses.shape[2]):
    #for j in range(1):    
        if (all(x == 0 for x in poses[:,0,j]) or all(y == 0 for y in poses[:,1,j])):
            print("invalid poses in ", folderpath)
            invalid_pose_counter += 1
        else:
            x0 = int(np.floor(np.min(poses[np.nonzero(poses[:,1,j]),1,j]))) - margin
            x1 = int(np.floor(np.max(poses[np.nonzero(poses[:,1,j]),1,j]))) + margin
            y0 = int(np.floor(np.min(poses[np.nonzero(poses[:,0,j]),0,j]))) - margin
            y1 = int(np.floor(np.max(poses[np.nonzero(poses[:,0,j]),0,j]))) + margin
        
            #make sure image borders stay in range
            im_size_tmp = Image.fromarray(scenes[j]).size
            x0 = max(0,x0)
            x1 = min(x1,im_size_tmp[0]-1)
            y0 = max(0,y0)
            y1 = min(y1,im_size_tmp[1]-1)
        
            im_tmp = scenes[j][x0:x1,y0:y1]

#             plt.imshow(im_tmp)
            x_dif = im_size_total[0] - x1 + x0
            y_dif = im_size_total[1] - y1 + y0
            x_l = np.floor(x_dif/2)
            x_r = np.ceil(x_dif/2)
            y_l = np.floor(y_dif/2)
            y_r = np.ceil(y_dif/2)
        
            im_pad = np.pad(im_tmp,[(int(x_l),int(x_r)),(int(y_l),int(y_r)),(0,0)],'constant')#,'constant', constant_values=((0, 0),(0,0)))

            #plt.imshow(im_pad)
            
            framename = '{:03d}'.format(j+int(annotations[''][0]))
            #print("TEST",parentfolder,anglepath,framename)
            saving_path = origin + 'preprocessing/crops/'+ person + '/' + sequence + '/' + sequence + '-' + angle + '/'
            filename = person + '-' + sequence + '-' + angle + '_frame_' + framename + '.jpg'
            totalpath = saving_path + filename
#             print("Image name", filename)
#             print("Total path", totalpath)
            Image.fromarray(im_pad).save(totalpath)
            #im_save.save(totalpath)
            
            #print("i",i,"j",j,"poses",poses[:,0,j],poses[:,1,j])
            #print("invalid path:",totalpath)
            #plt.imshow(im)
print("Invalid counter:",invalid_pose_counter)
print("Done")

            




In [ ]:
b = np.max([p for p in p_x if p != 0])
print (b)
print(len(p_x))
# [e for i, e in enumerate(p_x) if e != 0]


In [ ]:
print(y_max_list)